In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from scipy.stats import multivariate_normal
from functools import reduce

In [2]:
data = pd.read_csv('/Users/yuwenchen/Pokemon-Classifier/Pokemon.csv')
data = pd.DataFrame(data)
data.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [3]:
df = data[['Attack', 'Defense', 'Type 1']]

# all grass type pokemon 
grassType = df.loc[df['Type 1'] == 'Grass']
# all normal type pokemon
normalType = df.loc[df['Type 1'] == 'Normal']

# derive 50% grass type pokemon as training set(35)
grass_training = grassType.sample(frac=0.5, replace=False, random_state=1)
# derive 50% normal type pokemon as training set(49)
normal_training = normalType.sample(frac=0.5, replace=False, random_state=1)

# Generative Model
<br> Assume all the training set are sampled from [Multivariate Normal Distribution.](https://en.wikipedia.org/wiki/Multivariate_normal_distribution)

<br> Fucntion to get μ*

<br> assume 35 grass type pokemons are generated from the Gaussian Distrubution with the maximum likelihood. 

In [4]:
num_g = grass_training.shape[0]
num_n = normal_training.shape[0]

# assume 35 grass type pokemons are generated from the Gaussian Distrubution with the maximum likelihood. 


# transform the Attack and Defense attributes into 2 dimensional list
grass_list = grass_training.values.T.tolist()
normal_list = normal_training.values.T.tolist()

# caculate the μ*
u = 0
for i in range (num_g):
    x_n = np.array([[grass_list[0][i]],[grass_list[1][i]]])
    u = u + x_n

u_max = u/num_g
u_max = u_max.astype(int)

# caculate the ∑*
sig = 0
for i in range (num_g):
    # current
    x_n = np.array([[grass_list[0][i]],[grass_list[1][i]]])
    left = x_n - u_max
    right = left.T
    sig = sig + left * right

sig_max = sig/num_g
sig_max = sig_max.astype(int)

# Caculate P(C1) and P(C2)

In [50]:
# 35 grass type pokemon, 49 noaml type pokemon
# grass as Class 1, noaml as Class 2
p_c1 = num_g/(num_g + num_n)
p_c2 = num_n/(num_g + num_n)

# Caculate P(x|C1) and P(x|C2)

In [51]:
# get multivariate normal distribution
def getMeanCor(pokeList:list, length:int):
    
    # caculate the μ*
    u = 0
    for i in range (length):
        x_n = np.array([[pokeList[0][i]],[pokeList[1][i]]])
        u = u + x_n

    u_max = u/num_g
    u_max = u_max.astype(int)

    # caculate the ∑*
    sig = 0
    for i in range (length):
        # current
        x_n = np.array([[pokeList[0][i]],[pokeList[1][i]]])
        left = x_n - u_max
        right = left.T
        sig = sig + left * right

    sig_max = sig/num_g
    sig_max = sig_max.astype(int)
    
    u_max_flat = list(np.concatenate(u_max).flat)    
    
    return u_max_flat, sig_max

In [52]:
# grass as Class 1, noaml as Class 2
# get generative model
meanC1, sigC1 = getMeanCor(grass_list, num_g)
meanC2, sigC2 = getMeanCor(normal_list, num_n)

In [58]:
distC1 = multivariate_normal(mean=meanC1, cov=sigC1)
distC2 = multivariate_normal(mean=meanC2, cov=sigC2)

print(distC1.pdf([109,88]))

0.00014541909950428286


# Classification

In [56]:
def proba(w,x,y,z):
    return w*x / (w*x + y*z)

In [64]:
# grass type

test = [110,70]
p_C1_x =  distC1.pdf(test)*p_c1 / (distC1.pdf(test)*p_c1 + distC2.pdf(test)*p_c2)

In [63]:
normalType

,Attack,Defense,Type 1
20,45,40,Normal
21,60,55,Normal
22,80,75,Normal
23,80,80,Normal
24,56,35,Normal
25,81,60,Normal
26,60,30,Normal
27,90,65,Normal
44,45,20,Normal
45,70,45,Normal


In [65]:
p_C1_x

0.22062967667755184